# Importing modules

In [1]:
# !pip install tensorflow
!pip install prophet
!pip install openpyxl
!pip install joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 122.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 99.4 MB/s eta 0:00:00


In [2]:
# !pip install xgboost

In [3]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
import boto3
import s3fs
from config_module import Config
config = Config()

# **IMPORTANT**: Have to do this line *before* importing tensorflow
# os.environ['PYTHONHASHSEED'] = str(2)
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

from sklearn.preprocessing import MinMaxScaler
# import torch
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.models import save_model, model_from_json, Sequential
# from tensorflow.keras.layers import Dense, LSTM, Dropout
from pickle import dump, load
# from keras.models import load_model
import warnings
# from torch import nn
# import torch.nn.functional as F

import random
# def reset_random_seeds():
#     os.environ['PYTHONHASHSEED'] = str(2)
#     tf.random.set_seed(2)
#     np.random.seed(2)
#     random.seed(2)
#     tf.keras.utils.set_random_seed(2024)

# Make some random data
# reset_random_seeds()

# Set seed for reproducibility
np.random.seed(42)
# tf.random.set_seed(42)
pd.options.mode.chained_assignment = None

# Import modules
import itertools
import logging
import math
import pickle
from datetime import datetime
# import tarfile
import joblib
# import tempfile
import ast
import seaborn as sn
# import xgboost as xgb
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
from prophet import Prophet
from sklearn.model_selection import ParameterGrid
# from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error,
                             mean_squared_error)
from sklearn.model_selection import (GridSearchCV, TimeSeriesSplit,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
warnings.filterwarnings('ignore')
random.seed(12)

In [4]:
# Initialize the S3 filesystem
fs = s3fs.S3FileSystem()
s3_client = boto3.client('s3')
print(datetime.now())

2024-09-13 20:52:28.683229


In [5]:
start_time = datetime.now()

# Best fit for parts - Training and saving best fit parameters for each model

In [6]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    return data

In [7]:
def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*")
    
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [8]:
def get_latest_s3_file(bucket_name, path_prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given path_prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path_prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file path_prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{path_prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific path_prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

In [9]:
# Define hyperparameters to search over
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'yearly_seasonality': [True, False],
    # 'changepoint_prior_scale': [0.01, 0.07, 0.09],
    # 'seasonality_prior_scale': [0.1, 5.0, 10.0],
    'changepoint_prior_scale': [0.01, 0.05, 0.07],
    'seasonality_prior_scale': [0.1, 1.0, 3.0, 5.0],
    
}

In [10]:
# Specify your S3 bucket and path
bucket = config.get_value('bucket')
prefix_output_file = config.get_value('prefix_cleaned_training_file_path')
file_name_prefix = config.get_value('cleaned_training_filename_prefix')
file_path = get_latest_s3_file(bucket,prefix_output_file,file_name_prefix)

old_data = pd.read_excel('s3://'+f'{bucket}/{file_path}')
parts_list = old_data['part_number'].unique().tolist()

In [11]:
file_path

'persist-bo/persist-bo-demand-forecast/cleaned-training-data/clean_data_imputed_2024-09-13.xlsx'

In [12]:
old_data['part_number'].nunique()

6525

In [13]:
def calculate_mape(x, y):
    """
    Function to calculate the Mean Absolute Percentage Error (MAPE) between two values x and y.
    
    Parameters:
    x: The true value.
    y: The predicted value.
    
    Returns:
    The MAPE value as a float.
    """
    
    # Calculating MAPE for x={x} and y={y}")
    
    # Check if x is NaN
    if math.isnan(x):
        return 0
    
    # Check if x and y are equal
    if x == y:
        # x and y are equal. Returning MAPE as 1.")
        return 1
    
    # Check if x is zero
    elif x == 0:
        # x is zero. Returning MAPE as 0 to avoid division by zero.")
        return 0
    
    # Calculate MAPE
    else:
        mape = 1 - (abs(x - y) / x)
        return mape

In [14]:
def prophet_forecast(parts_data_period, parts_data_train, parts_data_test, hyperparameters, period_lag, m_path):
    """
    Perform a forecast using Prophet for multiple hyperparameter combinations.
 
    Args:
    - data (DataFrame): Input DataFrame containing actual values for multiple products.
    - hyperparameters (list of dict): List of dictionaries containing hyperparameter combinations.
 
    Returns:
    - DataFrame: DataFrame containing actuals, fitted values, and future predictions for each product and hyperparameter combination.
    """
    results = []
    
    # Apply Box-Cox transformation: Add 1 to avoid log(0), then take the natural log
    parts_data_train['Total_qty_per_month_transformed'] = np.log(parts_data_train['Total_qty_per_month'] + 1)
    
    for params in ParameterGrid(hyperparameters):
        model = Prophet(**params)
        
        # Fit the model using the transformed data
        model.fit(parts_data_train.rename(columns={'period': 'ds', 'Total_qty_per_month_transformed': 'y'}))
        
        future = model.make_future_dataframe(periods=period_lag, freq='MS')
        forecast = model.predict(future)
        
        # Reverse the Box-Cox transformation: Take exponent and subtract 1
        forecast['yhat_transformed'] = np.exp(forecast['yhat']) - 1
        
        sample_result = forecast[['ds', 'yhat_transformed']].rename(columns={'ds': 'period', 'yhat_transformed': 'fitted'})
        sample_result['part_number'] = parts_data_period['part_number'].iloc[0]
        sample_result['actual'] = parts_data_period['Total_qty_per_month'].values
        sample_result['type'] = None
        sample_result['type'].iloc[:-period_lag] = 'Train'
        sample_result['type'].iloc[-period_lag:] = 'Test'
        
        sample_result['model_name'] = str(params)
        results.append(sample_result)
        
        pn = parts_data_period['part_number'].iloc[0]
        
#         output_file = os.path.join(f"s3://{bucket}/{m_path}/model_folder", f"prophet-{pn}.joblib")
#         output_file_temp = f"/tmp/prophet-{pn}.joblib"
#         with fs.open(output_file, 'wb') as f:
#             joblib.dump(model, f)
#         with open(output_file_temp, 'wb') as model_file:
#             joblib.dump(model, model_file)

#         tar_model_path = output_file_temp
#         with tarfile.open(tar_model_path, "w:gz") as tar:
#             tar.add(output_file_temp, arcname=os.path.basename(output_file_temp))
#         # Upload the tar.gz archive to S3
#         s3_model_path = f"{m_path}/model_folder/prophet-{pn}.tar.gz"
#         s3_client.upload_file(tar_model_path, bucket, s3_model_path)
    
    return pd.concat(results, ignore_index=True)

In [15]:
def create_test_complete_dataset_from_scratch(df, start_date, end_date, pn):
    """
    Function to create a complete dataset from scratch for a given part number (pn) over a specified date range.
    
    Parameters:
    df (DataFrame): The input DataFrame (not used in this function but kept for consistency).
    start_date (datetime): The start date for the period.
    end_date (datetime): The end date for the period.
    pn (str): The part number for which the dataset is being created.
    
    Returns:
    DataFrame: A DataFrame with all months in the specified period and the specified part number.
    """
    
    # Converting start and end dates to strings: {start_date} to {end_date}
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    # Parsing start and end dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
     
    # Creating a DataFrame with all months in the specified period
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    all_months_df = pd.DataFrame(date_range, columns=['period'])
    all_months_df['period'] = all_months_df['period'].dt.strftime('%Y-%m')
    
    # Creating a DataFrame with the specified part number
    all_product_df = pd.DataFrame({'part_number': [pn]})
    
    # Cross-merging date DataFrame and part number DataFrame
    merged_df = all_months_df.merge(all_product_df, on=None, how='cross')
    merged_df['period'] = pd.to_datetime(merged_df['period'])
    
    # Completed creating the test complete dataset
    return merged_df

In [16]:
def create_test_complete_dataset(df, start_date, end_date):
    """
    Function to create a complete dataset with all months in the specified period for each part number in the input DataFrame.
    
    Parameters:
    df (DataFrame): The input DataFrame containing at least 'part_number' and 'period' columns.
    start_date (datetime): The start date for the period.
    end_date (datetime): The end date for the period.
    
    Returns:
    DataFrame: A DataFrame with all months in the specified period for each part number in the input DataFrame.
    """
    
    # Converting start and end dates to strings
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    # Parsing start and end dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Creating a DataFrame with all months in the specified period
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    all_months_df = pd.DataFrame(date_range, columns=['period'])
    all_months_df['period'] = all_months_df['period'].dt.strftime('%Y-%m')
    
    # Creating a DataFrame with all unique part numbers from the input DataFrame
    all_product_df = pd.DataFrame({'part_number': df['part_number'].unique()})
    
    # Cross-merging date DataFrame and part number DataFrame
    merged_df = all_months_df.merge(all_product_df, on=None, how='cross')
    merged_df['period'] = pd.to_datetime(merged_df['period'])
    
    # Merging the created DataFrame with the input DataFrame on 'period' and 'part_number'
    merged_df = merged_df.merge(df, left_on=['period', 'part_number'], right_on=['period', 'part_number'], how='left')
    
    # Completed creating the test complete dataset
    return merged_df

In [17]:
def create_complete_dataset(data_new, train_start_date, train_end_date):
    """
    Function to create training and testing datasets from the input data for the specified training period.
    
    Parameters:
    data_new (DataFrame): The input DataFrame containing at least 'part_number' and 'period' columns.
    train_start_date (str): The start date for the training period in 'YYYY-MM-DD' format.
    train_end_date (str): The end date for the training period in 'YYYY-MM-DD' format.
    
    Returns:
    tuple: A tuple containing two DataFrames: the training dataset and the testing dataset.
    """
    
    # Converting 'period' column to datetime format and formatting it as 'YYYY-MM-DD'
    data_new['period'] = pd.to_datetime(data_new['period']).dt.strftime('%Y-%m-%d')
    
    # Sorting the data by 'part_number' and 'period'
    data_new = data_new.sort_values(['part_number', 'period'])
    
    # Filtering the data from the specified training start date
    data_new = data_new[data_new['period'] >= train_start_date]
    
    # Converting 'period' column back to datetime format for further processing
    data_new['period'] = pd.to_datetime(data_new['period'])
    
    df_filtered_1 = data_new.copy()
    
    # Creating the training dataset
    df_train = df_filtered_1[df_filtered_1['period'] <= train_end_date]
    
    test_start_date = (pd.to_datetime(train_end_date) + pd.DateOffset(months=1)).strftime('%Y-%m-%d')
    test_end_date = (pd.to_datetime(train_end_date) + pd.DateOffset(months=3)).strftime('%Y-%m-%d')
    
    # Creating the testing dataset
    df_test = df_filtered_1[(df_filtered_1['period'] >= test_start_date) & (df_filtered_1['period'] <= test_end_date)]
    # logging.info(f"Testing data period: {df_test['period'].min()} to {df_test['period'].max()}")
    
    # Completed creating the training and testing datasets
    return df_train, df_test

In [18]:
def create_TS_model_prophet(df, hyp_dict, p, t_start_date, t_end_date, lag_data, model_url):
    """
    Function to create a time series model using Prophet, including handling for train-test split and missing test data.
    
    Parameters:
    df (DataFrame): The input DataFrame containing at least 'part_number' and 'period' columns.
    hyp_dict (dict): Dictionary containing hyperparameters for the Prophet model.
    p: Additional parameters required by the Prophet model.
    t_start_date (str): The start date for the training period in 'YYYY-MM-DD' format.
    t_end_date (str): The end date for the training period in 'YYYY-MM-DD' format.
    lag_data: Additional lag data required by the model.
    
    Returns:
    DataFrame: A DataFrame containing the Prophet model forecasts.
    """
    
    # Split data into training and testing sets
    
    # Extracting unique part number from the DataFrame
    pn = df['part_number'].unique().tolist()[0]
    
    # Creating a copy of the input DataFrame
    df_new = df.copy()
    
    # Splitting the data into training and testing sets
    train, test = create_complete_dataset(df_new, t_start_date, t_end_date)
    train_clean = train.copy()
    
    if test.empty:
        test = create_test_complete_dataset_from_scratch(test, 
                                                         train_clean['period'].max() + pd.DateOffset(months=1), 
                                                         train_clean['period'].max() + pd.DateOffset(months=3),
                                                         pn)
    
    if test.shape[0] < 3:
        test = create_test_complete_dataset(test, 
                                            train_clean['period'].max() + pd.DateOffset(months=1), 
                                            train_clean['period'].max() + pd.DateOffset(months=3))
    
    # Extracting date ranges for training and testing sets
    data_start_date = train['period'].min()
    train_end_date = train['period'].max()
    test_start_date = test['period'].min()
    data_end_date = test['period'].max()
    
    # Combining training and testing data for the period data
    period_data = pd.concat([train_clean, test], ignore_index=True)
    
    # Forecasting using the Prophet model
    prophet_all_df = prophet_forecast(period_data, train_clean, test, hyp_dict, p, model_url)
    
    return prophet_all_df

In [19]:
def check_input_type(input_value):
    """
    Function to check the type of the input value and convert it to a string if it is an integer.
    
    Parameters:
    input_value: The value to be checked and possibly converted.
    
    Returns:
    The input value converted to a string if it is an integer, otherwise the original input value.
    """
    
    # Checking the type of the input value
    if isinstance(input_value, int):
        # Input value is of type int: {input_value}. Converting to string.
        return str(input_value)
    else:
        return input_value

In [20]:
def extract_metrics(parts_df_seg, data_set, train_start_date, train_end_date, lag_val, model_path_url):
    """
    Function to extract metrics using time series modeling (Prophet) for each part number in the input DataFrame segment.
    
    Parameters:
    parts_df_seg (list): List of part numbers to process.
    data_set (DataFrame): The input dataset containing at least 'part_number' and 'period' columns.
    train_start_date (str): The start date for the training period in 'YYYY-MM-DD' format.
    train_end_date (str): The end date for the training period in 'YYYY-MM-DD' format.
    lag_val: Additional lag value required for modeling.
    
    Returns:
    DataFrame: A DataFrame containing metrics extracted using time series modeling for each part number.
    """
    
    final_best_df_data = pd.DataFrame()
    index_val = 0
    
    for parts in parts_df_seg:
        try:
            
#             print(f"Processing part number: {parts}: {index_val}")
            parts = check_input_type(parts)
            # Converted part number to string
            
            # Filtering data for part number
            parts_data = data_set[data_set['part_number'] == parts]
    
            # Creating time series model (Prophet) for part number
            prophet_data = create_TS_model_prophet(parts_data, param_grid, 3, train_start_date, train_end_date, lag_val, model_path_url)
            
            # Pivoting Prophet model data for part number
            pivot_data_prophet = prophet_data.pivot(index='period', columns='model_name', values='fitted')
            
            # Merging Prophet model data for part number
            merged_data_prophet = prophet_data.drop(['fitted', 'model_name'], axis=1).drop_duplicates().merge(pivot_data_prophet, how='left', on='period')
            
            # Concatenating data for part number
            final_best_df_data = pd.concat([final_best_df_data, merged_data_prophet], ignore_index=True)
            
            index_val += 1
        except Exception as e:
            print(f"Error processing part number {parts}: {e}")
    print("Completed extracting metrics for all parts")
    return final_best_df_data

In [21]:
def create_bestfit_prophet_train(list_parts_all, main_data, start_date_train, end_date_train, lag_val, model_path):
    """
    Function to create the best fit Prophet model for training data and output the results to an Excel file.

    Parameters:
    list_parts_all (list): List of part numbers to process.
    main_data (DataFrame): Main dataset containing all necessary columns including 'part_number' and 'period'.
    start_date_train (str): Start date of the training period in 'YYYY-MM-DD' format.
    end_date_train (str): End date of the training period in 'YYYY-MM-DD' format.
    lag_val (int): Lag value used for modeling.
    output_path_folder (str): Folder path where the output Excel file will be saved.
    prophet_bestfit_filename (str): file name to store bestfit
    all_prophet_fitted_forecast_filename (str): file name to store all prophet_results

    Returns:
    None
    """

    # Starting best fit Prophet model creation and evaluation
    # Extract metrics for all parts
    parts_200_df = extract_metrics(list_parts_all, main_data, start_date_train, end_date_train, lag_val, model_path)
    
    
    # Calculate MAPE for each model and store it in parts_200_df_mape
    parts_200_df_mape = parts_200_df.iloc[:,:4]
    for cols in parts_200_df.iloc[:,4:].columns:
        parts_200_df_mape[cols] = parts_200_df.apply(lambda row: calculate_mape(row['actual'], row[cols]), axis=1)
    
    # Calculate average MAPE for each model
    mape_df = pd.DataFrame()
    for model_name in parts_200_df_mape.columns[4:].tolist():
        model_mape_df = parts_200_df_mape[parts_200_df_mape['type']=='Test'].groupby(['part_number'])[model_name].mean().reset_index()
        model_mape_df = model_mape_df.melt(id_vars='part_number', var_name=model_name, value_vars=model_name)
        model_mape_df.columns = ['part_number', 'model', 'value']
        mape_df = pd.concat([mape_df, model_mape_df], ignore_index=True)
    
    # Select the model with the highest MAPE for each part number
    df = mape_df.loc[mape_df.groupby('part_number')['value'].idxmax()].reset_index(drop=True)
    
    # Filter final best fit data for 'Test' type
    final_best_df_subset = parts_200_df[parts_200_df['type']=='Test']
    
    # Prepare final DataFrame with best fit Prophet model results
    final_df_best_fit = pd.DataFrame()
    for pnt in df['part_number'].unique():
        best_model_name = df[df['part_number']==pnt]['model'].values[0]
        forecasted_data_subset_pnt = final_best_df_subset.loc[final_best_df_subset['part_number']==pnt, 
                                                                ['part_number', 'period', 
                                                                 'type', 'actual', best_model_name]]
        forecasted_data_subset_pnt['model_name'] = best_model_name
        forecasted_data_subset_pnt.rename(columns={best_model_name:'forecasted_values'}, inplace=True)
        final_df_best_fit = pd.concat([final_df_best_fit, forecasted_data_subset_pnt], ignore_index=True)
    
    # Calculate accuracy prediction based on actual and forecasted values
    final_df_best_fit = final_df_best_fit.assign(accuracy_pred = lambda x: np.where(x['actual']!=0,
                                                                                    (1-(abs(x['forecasted_values'] - x['actual'])/x['actual']))*100,
                                                                                    0))
    final_df_best_fit['accuracy_pred'].fillna(0, inplace=True)
    
    return final_df_best_fit, parts_200_df


In [22]:
train_start_date = (pd.to_datetime(old_data['period']).min()).strftime('%Y-%m-%d')
train_end_date = (pd.to_datetime(old_data['period']).max()+relativedelta(months=-3)).strftime('%Y-%m-%d')

In [23]:
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").disabled=True

In [24]:
print(datetime.now())

2024-09-13 20:52:47.150389


In [25]:
print(train_start_date, train_end_date)

2021-01-01 2024-03-01


In [26]:
prefix_output = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}"

prefix_output_prophet_model = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'prophet'}"

In [27]:
prefix_output_prophet_model

'persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/prophet'

In [28]:
parts_list_updated = parts_list[:1000]
# parts_list_updated = parts_list_updated[:100]    # line to be commented post testing
prophet_trained_bestfit_data, full_data_prophet = create_bestfit_prophet_train(parts_list_updated, old_data, train_start_date, train_end_date, 0, prefix_output_prophet_model)

Completed extracting metrics for all parts


In [29]:
print(datetime.now())

2024-09-14 19:57:42.359287


In [30]:
prophet_trained_bestfit_data

,part_number,period,type,actual,forecasted_values,model_name,accuracy_pred
0,04000081B0,2024-04-01,Test,8.0,7.178818,"{'changepoint_prior_scale': 0.01, 'seasonality...",89.735221
1,04000081B0,2024-05-01,Test,2.0,5.894491,"{'changepoint_prior_scale': 0.01, 'seasonality...",-94.724546
2,04000081B0,2024-06-01,Test,3.0,3.135172,"{'changepoint_prior_scale': 0.01, 'seasonality...",95.494282
3,0411106101,2024-04-01,Test,0.0,16.051309,"{'changepoint_prior_scale': 0.01, 'seasonality...",0.000000
4,0411106101,2024-05-01,Test,0.0,5.613132,"{'changepoint_prior_scale': 0.01, 'seasonality...",0.000000
...,...,...,...,...,...,...,...
19570,PZ08100005,2024-05-01,Test,156.0,132.441210,"{'changepoint_prior_scale': 0.01, 'seasonality...",84.898212
19571,PZ08100005,2024-06-01,Test,153.0,152.967751,"{'changepoint_prior_scale': 0.01, 'seasonality...",99.978923
19572,PZT0038023,2024-04-01,Test,320.0,239.090712,"{'changepoint_prior_scale': 0.05, 'seasonality...",74.715848
19573,PZT0038023,2024-05-01,Test,305.0,327.156868,"{'changepoint_prior_scale': 0.05, 'seasonality...",92.735453


In [31]:
training_date = pd.to_datetime(config.get_value('model_training_date'))
file_date = training_date.strftime("%Y-%m-%d")

In [32]:
# Specify your S3 bucket and path
# prefix_output = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}"
# file_name_prophet_train = f'{config.get_value("file_name_prophet_train")}{datetime.now().strftime("%Y-%m-%d")}.xlsx'
file_path_bestfit_prophet = f"s3://{bucket}/{prefix_output}/{config.get_value('file_name_prophet_train')}{file_date}.xlsx"

# file_name_all_prophet_config_output = f'{config.get_value("file_name_all_prophet_config_output")}{datetime.now().strftime("%Y-%m-%d")}.xlsx'
# file_path_all_prophet_config_output = f's3://{bucket}/{prefix_output_prophet_data}/{file_name_all_prophet_config_output}'



def generate_data(prophet_bestfit_trained, prophet_all_data):
    prophet_bestfit_trained.to_excel(file_path_bestfit_prophet, index=False)
#     prophet_all_data.to_excel(file_path_all_prophet_config_output, index=False)

generate_data(prophet_trained_bestfit_data, full_data_prophet)

### Generate best fit prophet configs

In [33]:
parts_bestconfig_prophet = prophet_trained_bestfit_data[['part_number', 'model_name']].drop_duplicates().reset_index(drop=True)
# prefix_output_prophet_model = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'prophet'}"
filename_bestfit_prophet_config = config.get_value('filename_bestfit_prophet_config')
filepath_prophet_config = f"s3://{bucket}/{prefix_output_prophet_model}/{filename_bestfit_prophet_config}{file_date}.xlsx"
def generate_prophet_config(prophet_config_data, path_to_store):
    prophet_config_data.to_excel(path_to_store, index=False)

generate_prophet_config(parts_bestconfig_prophet, filepath_prophet_config)

In [34]:
end_time = datetime.now()
print("Total time taken for prophet training::", (end_time-start_time).seconds/60, " minutes")

Total time taken for prophet training:: 1385.3  minutes
